select all customers with customer_id not null, merge_flag is flase, gender male and female

In [0]:
df = spark.sql("select * from bronze.customer where customer_id is not null and merge_flag = false and gender in ('Male', 'Female')")
display(df)

remove rows where registration_date > dob

In [0]:
from pyspark.sql.functions import col
df_dob_check = df.filter(col("registration_date") < col("date_of_birth"))

display(df_dob_check)

merge to silver layer

In [0]:
df_dob_check.createOrReplaceTempView("transformed_customer")
spark.sql("MERGE INTO silver.customer t USING transformed_customer s ON t.customer_id = s.customer_id WHEN MATCHED THEN UPDATE SET t.first_name = s.first_name, t.last_name = s.last_name, t.email = s.email, t.phone = s.phone, t.country = s.country, t.registration_date = s.registration_date, t.date_of_birth = s.date_of_birth, t.gender = s.gender, t.merged_timestamp = current_timestamp() WHEN NOT MATCHED THEN INSERT (customer_id, first_name, last_name, email, phone, country, city, registration_date, date_of_birth, gender, merged_timestamp) VALUES (s.customer_id, s.first_name, s.last_name, s.email, s.phone, s.country, s.city, s.registration_date, s.date_of_birth, s.gender, current_timestamp())")

In [0]:
spark.sql("update bronze.customer set merge_flag = true where merge_flag = false")